In [8]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [3]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percent: float
    summary: str

In [12]:
def calculate_sr(state: BatsmanState) -> dict:
    runs = state['runs']
    balls = state['balls']

    sr = runs/balls * 100

    return {'sr': sr}

def calculate_bpb(state: BatsmanState) -> dict:
    fours = state['fours']
    sixes = state['sixes']
    balls = state['balls']

    bpb =  balls/(fours+sixes)

    return {'bpb':bpb}

def calculate_bp(state: BatsmanState) -> dict:      
    fours = state['fours']
    sixes = state['sixes']
    runs = state['runs']

    boundary_percent = ((fours*4)+(sixes*6))/runs

    return {"boundary_percent":boundary_percent}


def summary(state: BatsmanState) -> BatsmanState:
    summary = f"""
    Strike Rate - {state['sr']} \n
    Balls per boundary - {state['bpb']}\n
    Boundary Percent - {state['boundary_percent']}
    """

    state['summary'] = summary

    return state

In [13]:
graph = StateGraph(BatsmanState)

graph.add_node("calculate_sr",calculate_sr)
graph.add_node("calculate_bpb",calculate_bpb)
graph.add_node("calculate_bp",calculate_bp)
graph.add_node("summary",summary)


#edges

graph.add_edge(START,"calculate_sr")
graph.add_edge(START,"calculate_bpb")
graph.add_edge(START,"calculate_bp")

graph.add_edge("calculate_sr","summary")
graph.add_edge("calculate_bpb","summary")
graph.add_edge("calculate_bp","summary")

graph.add_edge("summary",END)


workflow = graph.compile()



In [18]:
initial_state = {"runs":143,"balls":78,"fours":13,"sixes":10}

final_state = workflow.invoke(initial_state)

In [19]:
print(final_state['summary'])


    Strike Rate - 183.33333333333331 

    Balls per boundary - 3.391304347826087

    Boundary Percent - 0.7832167832167832
    
